In [2]:
import os
import numpy as np
import glob
import sys
import subprocess
import argparse
import torch

/home/rchoudhu/.conda/envs/agentformer/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
data_root  = "./datasets/tepper/Pedestrian_labels"

seq_name = '0'

label_path = os.path.join(data_root, '{}_frame.txt'.format(seq_name))
raw_data = np.genfromtxt(label_path, delimiter=',')

In [3]:
# Don't do any scaling yet.
# only pedestrians.
gt = raw_data
frames = gt[:, 0].astype(np.float32).astype(int)
fr_start, fr_end = frames.min(), frames.max()
init_frame = fr_start
num_fr = fr_end + 1 - fr_start
past_frames = 8
fut_frames = 12
frame_skip = 1
min_past_frames = 8
min_future_frames = 12 

In [4]:
def get_history_data(frame_idx):
    data_list = []
    for i in range(past_frames):
        if frame_idx - i < init_frame:
            data = []
        data = gt[gt[:, 0] == (frame_idx - i) * frame_skip]
        data_list.append(data)
    return data_list

In [5]:
def get_future_data(frame_idx):
    data_list = []
    for i in range(1, fut_frames + 1):
        data = gt[gt[:, 0] == frame_idx  + i * frame_skip]
        data_list.append(data)
    return data_list

In [6]:
def get_id(data):
    id = []
    for i in range(data.shape[0]):
        id.append(data[i, 1].copy())
    return id

def get_valid_id(pre_data, fut_data):
    cur_id = get_id(pre_data[0])
    valid_id = []
    for idx in cur_id:
        exist_pre = [(False if isinstance(data, list) else (idx in data[:, 1])) for data in pre_data[:min_past_frames]]
        exist_fut = [(False if isinstance(data, list) else (idx in data[:, 1])) for data in fut_data[:min_future_frames]]
        if np.all(exist_pre) and np.all(exist_fut):
            valid_id.append(idx)
    return valid_id

In [20]:
def get_history_motion(data_list, valid_id):
    motion = []
    mask = []
    for identity in valid_id:
        mask_i = torch.zeros(past_frames)
        past_coords = torch.zeros(past_frames, 3)
        for frame_idx in range(past_frames):
            past_data = data_list[frame_idx]
            if len(past_data) > 0 and identity in past_data[:, 1]:
                # Keep all indices (x, y, z), don't apply traj scale.
                found_data = past_data[past_data[:, 1] == identity].squeeze()
                past_coords[past_frames-1-frame_idx, :] = torch.from_numpy(found_data[[2,3,4]]).float()
                mask_i[past_frames - 1 - frame_idx] = 1.0
            elif frame_idx > 0:
                past_coords[past_frames-1 - frame_idx, :] = past_coords[past_frames - frame_idx, :]
            else:
                raise ValueError('current id missing in the first frame!')
        motion.append(past_coords)
        mask.append(mask_i)
    
    return motion, mask

def get_future_motion(data_list, valid_id):
    motion = []
    mask = []
    for identity in valid_id:
        mask_i = torch.zeros(fut_frames)
        mask_i = torch.zeros(fut_frames)
        pos_3d = torch.zeros([fut_frames, 3])
        for j in range(fut_frames):
            fut_data = data_list[j]              # cur_data
            if len(fut_data) > 0 and identity in fut_data[:, 1]:
                found_data = fut_data[fut_data[:, 1] == identity].squeeze()[[2,3,4]]
                pos_3d[j, :] = torch.from_numpy(found_data).float()
                mask_i[j] = 1.0
            elif j > 0:
                pos_3d[j, :] = pos_3d[j - 1, :]    # if none, copy from previous
            else:
                raise ValueError('current id missing in the first frame!')
        motion.append(pos_3d)
        mask.append(mask_i)
    
    return motion, mask

In [21]:
def get_data(frame_idx):
    assert frame_idx - init_frame >= 0 and frame_idx - init_frame < num_fr

    pre_data = get_history_data(frame_idx)
    fut_data = get_future_data(frame_idx)
    valid_id = get_valid_id(pre_data, fut_data)

    if len(pre_data[0]) == 0 or len(fut_data[0]) == 0 or len(valid_id) == 0:
        return None
    pred_mask = None
    heading = None
    
    pre_motion_3D, pre_motion_mask = get_history_motion(pre_data, valid_id)
    fut_motion_3D, fut_motion_mask = get_future_motion(fut_data, valid_id)

    data = {
        'pre_motion_3D': pre_motion_3D,
        'fut_motion_3D': fut_motion_3D,
        'fut_motion_mask': fut_motion_mask,
        'pre_motion_mask': pre_motion_mask,
        'pre_data': pre_data,
        'fut_data': fut_data,
        'heading': heading,
        'valid_id': valid_id,
        'pred_mask': pred_mask,
        'scene_map': None,
        'seq': seq_name,
        'frame': frame_idx
    }

    return data


In [1]:
# Play with H36M
import cdflib
s1_root = "/home/rchoudhu/research/pose_forecasting/datasets/h3.6m/S1/MyPoseFeatures/D3_Positions"
path = os.path.join(s1_root, "Posing 1.cdf")
hf = cdflib.CDF(path)

In [13]:
h36m_data = np.load("./datasets/h3.6m/data_3d_h36m.npz", allow_pickle=True)

In [17]:
class DatasetH36M():
    # we need to make this a file.
    def __init__(self, mode, t_his=25, t_pred=100, actions='all', use_vel=False):
        self.use_vel = use_vel
        super().__init__(mode, t_his, t_pred, actions)
        if use_vel:
            self.traj_dim += 3

    def prepare_data(self):
        self.data_file = os.path.join('datasets', 'h3.6m', 'data_3d_h36m.npz')
        self.subjects_split = {'train': [1, 5, 6, 7, 8],
                               'test': [9, 11]}
        self.subjects = ['S%d' % x for x in self.subjects_split[self.mode]]
        #self.skeleton = Skeleton(parents=[-1, 0, 1, 2, 3, 4, 0, 6, 7, 8, 9, 0, 11, 12, 13, 14, 12,
        #                                  16, 17, 18, 19, 20, 19, 22, 12, 24, 25, 26, 27, 28, 27, 30],
        #                         joints_left=[6, 7, 8, 9, 10, 16, 17, 18, 19, 20, 21, 22, 23],
        #                         joints_right=[1, 2, 3, 4, 5, 24, 25, 26, 27, 28, 29, 30, 31])
        self.removed_joints = {4, 5, 9, 10, 11, 16, 20, 21, 22, 23, 24, 28, 29, 30, 31}
        self.kept_joints = np.array([x for x in range(32) if x not in self.removed_joints])
        #self.skeleton.remove_joints(self.removed_joints)
        #self.skeleton._parents[11] = 8
        #self.skeleton._parents[14] = 8
        self.process_data()

    def process_data(self):
        data_o = np.load(self.data_file, allow_pickle=True)['positions_3d'].item()
        data_f = dict(filter(lambda x: x[0] in self.subjects, data_o.items()))
        if self.actions != 'all':
            for key in list(data_f.keys()):
                data_f[key] = dict(filter(lambda x: all([a in x[0] for a in self.actions]), data_f[key].items()))
                if len(data_f[key]) == 0:
                    data_f.pop(key)
        for data_s in data_f.values():
            for action in data_s.keys():
                seq = data_s[action][:, self.kept_joints, :]
                if self.use_vel:
                    v = (np.diff(seq[:, :1], axis=0) * 50).clip(-5.0, 5.0)
                    v = np.append(v, v[[-1]], axis=0)
                seq[:, 1:] -= seq[:, :1]
                if self.use_vel:
                    seq = np.concatenate((seq, v), axis=1)
                data_s[action] = seq
        self.data = data_f

In [18]:
ds = DatasetH36M()

TypeError: __init__() missing 1 required positional argument: 'mode'